<a href="https://colab.research.google.com/github/Solo7602/MADPA/blob/main/Mosh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"  # избегаем конфликта с TensorFlow

import pandas as pd
import numpy as np
import re
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tqdm import tqdm
import xgboost as xgb

# Загружаем данные
data = pd.read_csv('10k_dataset_processed_final.csv').head(100)
data['Comment'] = data['Comment'].fillna("")

# Создание возрастных групп
bins = [0, 18, 30, 45, 100]
labels = [0, 1, 2, 3]
data['age_group'] = pd.cut(data['Age'], bins=bins, labels=labels).astype(int)

# Числовые признаки
def count_punctuation(text):
    return len(re.findall(r"[.,!?;:\-–—]", text))

def ends_with_punctuation(text):
    text = text.strip()
    return int(text[-1] in {'.', ',', '!', '?'}) if text else 0

data['emoji_count'] = data['Comment'].apply(lambda x: len(re.findall(r'[\U0001F600-\U0001F64F]', x)))
data['punctuation_count'] = data['Comment'].apply(count_punctuation)
data['word_count'] = data['Comment'].apply(lambda x: len(x.split()))
data['avg_word_length'] = data['Comment'].apply(lambda x: np.mean([len(w) for w in x.split()]) if x.split() else 0)
data['ends_with_punct'] = data['Comment'].apply(ends_with_punctuation)
data['Comment_Length'] = data['Comment'].apply(len)

numeric_features = ['emoji_count', 'punctuation_count', 'word_count', 'avg_word_length', 'ends_with_punct', 'Comment_Length']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[numeric_features])

# BERT эмбеддинги
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

bert_embeddings = []
for comment in tqdm(data['Comment'], desc="BERT embedding"):
    try:
        emb = get_bert_embedding(comment)
    except Exception:
        emb = np.zeros(768)
    bert_embeddings.append(emb)
    print(emb)

bert_features = np.vstack(bert_embeddings)

# Объединение признаков
X = np.hstack([bert_features, scaled_features])

y = data['age_group'].values

# # Разделение данных
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Обучение XGBoost
# xgb_clf = xgb.XGBClassifier(
#     objective="multi:softmax",
#     num_class=4,
#     max_depth=6,
#     learning_rate=0.1,
#     n_estimators=200,
#     use_label_encoder=False,
#     eval_metric="mlogloss",
#     random_state=42
# )

# xgb_clf.fit(X_train, y_train)

# # Предсказание и оценка
# y_pred = xgb_clf.predict(X_test)
# print(classification_report(y_test, y_pred))

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
BERT embedding:   1%|          | 1/100 [00:00<00:21,  4.53it/s]

[-1.07810855e-01  3.37959677e-02 -1.54532388e-01  4.59664837e-02
  2.10242167e-01  1.66257158e-01  4.61089425e-02  3.53401989e-01
 -8.34905878e-02  9.06128362e-02  2.61318684e-01 -1.90474287e-01
 -3.51408452e-01 -2.84044333e-02 -6.41615868e-01 -2.63740540e-01
 -3.21303345e-02 -4.68573980e-02 -2.63602566e-02  3.28075886e-01
  3.03217322e-01 -8.61233547e-02 -1.48715982e-02  2.10626386e-02
 -1.13614485e-01  2.72272099e-02  8.77713505e-03  1.63819849e-01
  2.26149648e-01  2.65290737e-02  8.86771157e-02  2.25588575e-01
 -2.69575231e-02  4.55155745e-02 -1.33077845e-01 -3.15137133e-02
 -1.97533238e+00  4.04960103e-02 -1.16407059e-01  1.71356753e-01
 -3.58759642e-01 -1.52766347e-01  1.07429713e-01  1.57986954e-01
  9.55418795e-02  1.01330054e+00 -8.23320728e-03  4.74952549e-01
  1.18379855e+00  2.84837991e-01  3.54526877e-01 -1.58870518e-01
 -2.30406627e-01 -1.58198380e+00  1.06852956e-01 -9.64870974e-02
 -3.75521965e-02 -1.89271808e-01  4.76339161e-02  5.47952615e-02
  2.31212705e-01  4.61730

BERT embedding:   2%|▏         | 2/100 [00:00<00:23,  4.10it/s]

[ 1.52222663e-01  2.21916921e-02 -2.22619787e-01  8.24609622e-02
  6.45134002e-02  1.33856207e-01  3.54676158e-03  1.07872128e-01
  6.46877214e-02  6.60106242e-02  1.26463413e-01  2.49446034e-02
 -2.38109976e-01  1.11806616e-01 -6.24485970e-01 -1.80101618e-02
 -1.60940722e-01  1.04811221e-01  7.70218968e-02  3.55221070e-02
  6.78519160e-02  1.16980769e-01 -3.92106734e-02  4.55316249e-03
 -7.88965523e-02 -2.05646409e-03  1.71804041e-01 -2.76110116e-02
  9.27924737e-02  6.33526891e-02  8.62104818e-03  1.09187856e-01
 -4.75916825e-03 -2.56632809e-02  2.14557424e-02 -1.38666779e-01
 -2.39921331e+00  1.32561043e-01 -1.10253215e-01  7.68001676e-02
 -1.61837161e-01 -7.52094015e-02 -4.81705703e-02  5.28109223e-02
  5.99355064e-02  1.25331521e+00 -1.40070580e-02 -5.65048084e-02
  1.55791938e+00 -3.53224911e-02  7.04019591e-02 -2.84587085e-01
 -9.95715633e-02 -1.68227422e+00  6.06169440e-02  1.16243750e-01
 -6.57283440e-02  6.39188215e-02 -7.10476637e-02  3.12864631e-01
  5.09016551e-02  1.21198

BERT embedding:   3%|▎         | 3/100 [00:00<00:25,  3.83it/s]

...

[-1.54664919e-01 -8.15777406e-02 -3.43839347e-01  1.53560070e-02
  1.35311276e-01  6.94623664e-02  3.17816854e-01  3.43811959e-02
  2.43280772e-02  4.96594720e-02  1.49435222e-01 -1.49056306e-02
 -8.84002075e-02 -3.17775682e-02 -6.23482406e-01  3.93970944e-02
 -6.41676262e-02  2.22637713e-01  1.69914477e-02  8.29155836e-03
 -1.58228040e-01  5.22499643e-02 -5.69473915e-02  1.19157992e-01
 -3.48619297e-02 -3.14121604e-01  3.90000269e-02 -1.82542205e-02
  1.35587878e-03 -3.27146389e-02 -1.88649565e-01  2.61683166e-01
  1.66693479e-01 -4.88142855e-02  2.31253907e-01 -1.79925617e-02
 -2.30767798e+00  2.31390782e-02 -2.96723723e-01 -5.07962368e-02
 -1.10379368e-01 -1.11284815e-02  2.85570268e-02  2.47020926e-02
 -3.67202237e-02  1.31567442e+00  2.44881921e-02 -9.30386875e-03
  1.59126937e+00  1.24634683e-01  1.87678084e-01 -3.90327930e-01
  9.34801698e-02 -1.68752027e+00  2.86653996e-01  3.54186073e-02
 -1.38083771e-02 -1.08195819e-01 -1.08987302e-01  2.13134319e-01
  1.01709485e-01  3.81340

BERT embedding:  97%|█████████▋| 97/100 [00:12<00:00, 11.35it/s]

[ 1.94320176e-02  3.73917595e-02 -8.45735967e-02  7.00055584e-02
  1.15316883e-01  3.89920808e-02  4.24215682e-02  5.57607785e-02
  8.28695074e-02  5.56685328e-02  7.99198821e-03 -8.29287544e-02
 -2.64132321e-01 -1.30123213e-01 -4.87074584e-01 -1.06284264e-02
 -7.27921426e-02  1.56113103e-01 -1.88769400e-02  8.79640803e-02
  1.77483149e-02  1.81718573e-01 -2.81953871e-01  1.20155282e-01
 -3.13903876e-02 -1.90062582e-01  1.32555813e-01  6.11832067e-02
  1.75885841e-01 -3.40617709e-02 -9.17869955e-02  9.91219357e-02
  6.87464699e-02  1.55690429e-03  1.34318545e-01 -1.49948327e-02
 -2.28936696e+00 -1.27254784e-01 -1.12532675e-01 -8.17173719e-02
 -6.26741648e-02 -9.25755724e-02  5.11702374e-02  3.17989178e-02
 -6.59224838e-02  1.22931385e+00  4.88953143e-02  9.43285376e-02
  1.49865830e+00  7.51651525e-02  2.70622075e-01 -2.38952890e-01
  1.05424613e-01 -1.60613573e+00  2.26192549e-01  1.87952325e-01
 -1.81047022e-02 -1.05943330e-01 -2.93626431e-02  3.31350178e-01
 -2.06529275e-02  2.58638

BERT embedding: 100%|██████████| 100/100 [00:12<00:00,  7.93it/s]

[ 1.93779841e-02  1.17270947e-01 -1.75401866e-01  9.32560116e-02
  2.91016281e-01 -1.91542655e-02 -6.57549873e-02  5.05437329e-02
 -1.41293528e-02  8.88606235e-02  2.27228016e-01 -8.12276527e-02
 -1.63747165e-02 -7.13649765e-02 -5.39346814e-01 -1.33068878e-02
 -6.55205101e-02  1.43137306e-01 -7.96240754e-03 -6.15148507e-02
  1.45328104e-01  4.69377730e-03 -1.97788149e-01  7.93830976e-02
 -1.38501078e-01 -1.58913657e-02  1.94038779e-01  8.14039260e-02
  1.01577565e-01  1.16251446e-01  6.21481538e-02  1.61378488e-01
 -7.97054265e-03  1.15752794e-01  1.03890318e-02 -8.90445188e-02
 -2.21344829e+00 -3.53320949e-02 -1.54865980e-01  4.87229750e-02
 -2.73851514e-01 -1.12112230e-02 -1.28699645e-01 -2.75315847e-02
  5.05569465e-02  1.20026135e+00 -9.89563242e-02  1.98996902e-01
  1.46718657e+00  1.93949882e-02  4.89893034e-02 -1.55710980e-01
 -2.56098747e-01 -1.68516004e+00  5.56872003e-02  1.88003346e-01
 -9.18838829e-02 -7.76703060e-02  6.71038628e-02  4.17532586e-02
  8.32229778e-02  1.53345

In [ ]:
pip install gensim python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol